# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath,
# and join the file path and roots with the subdirectories using glob

for root, dirs, files in os.walk(filepath):    
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/Users/jhonsen/Documents/DS/UDACITY_DEND/project_2_Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Building Apache Cassandra tables 
 The newly created csv file (**event_datafile_new.csv**) should contain the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of the denormalized data in **event_datafile_new.csv**

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting Keyspace

In [7]:
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

## Creating 3 queries to model the data and create the corresponding tables

### Query 1.
> *Output*: artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

**Setting up primary key and columns**
- [x] to have unique sessions, composite key (`sessionId` and `itemInSession`) was used, as multiple songs can play during each session for each user.  
- [x] to match the query output, `artist`, `song`, `length` were used as columns   

In [8]:
query1 = "CREATE TABLE IF NOT EXISTS music_history "
query1 = query1 + "(sessionId int, itemInSession int, artist text, song text, length double, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO music_history (sessionId, itemInSession, artist, song, length) "
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query1, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))

#### Verifying that the data have been inserted into each table

In [10]:
query1= """SELECT artist, song, length
FROM music_history
WHERE sessionId=338 AND itemInSession=4;
"""

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.length)    
    

Faithless Music Matters (Mark Knight Dub) 495.3073


### Query 2.
> *Output*: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182  

**Setting up primary key and columns**.
- [x] To identify unique users, Composite (`sessionId` and `userid`) were chosen as Partition key. 
- [x] To determine sorting order, `itemInSession` was chosen as clustering column 
- [x] `artist`,`song`,`user` were used as columns, to match the query output

In [11]:
query2 = "CREATE TABLE IF NOT EXISTS user_music_history "
query2 = query2 + "(sessionId int, userid int, itemInSession int, artist text, song text, user text, \
                        PRIMARY KEY ((sessionId, userid), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC)"
try:
    session.execute(query2)
except Exception as e:
    print(e)
                    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_music_history (sessionId, userid, itemInSession, artist, song, user) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], (line[1]+" "+line[4])))

#### Verifying that the data have been inserted into each table

In [12]:
query2= """SELECT artist, song, user
FROM user_music_history
WHERE sessionId=182 AND userid=10;
"""

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.user)    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3.
> *Output*: every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'    

**Setting up primary key and columns**.
- [x]  `song` and `userid` were chosen as Partition key and Clustering column, respectively. `userid` was used as clustering column to prevent `user` confusion, as it is possible to have 2 users with identical firstName and LastName. 
- [x] `user` was used as column to match the query output

In [13]:
query3 = "CREATE TABLE IF NOT EXISTS user_song_history "
query3 = query3 + "(song text, userid int, user text,  PRIMARY KEY ((song), userid))"
try:
    session.execute(query3)
except Exception as e:
    print(e)
                    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song_history (song, userid, user) "
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), (line[1]+" "+line[4])))

#### Verifying that the data have been inserted into each table

In [14]:
query3= """SELECT user
FROM user_song_history
WHERE song='All Hands Against His Own';
"""

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.user)    

Jacqueline Lynch
Tegan Levine
Sara Johnson


---

### Dropping tables before closing out the sessions

In [15]:
query = "drop table music_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_music_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Closing the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()

---